In [1]:
import pandas as pd
import ast
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
import plotly.io as pi

In [2]:
# Loading dataset
metadata_path = "C:/Users/Neeraj kalantri/Downloads/cleaned_dataset/metadata.csv"
metadata_df = pd.read_csv(metadata_path)
metadata_df.head()

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
1,impedance,[2010. 7. 21. 16. 53. ...,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
2,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,NaN,NaN,NaN
3,impedance,[2010 7 21 20 31 5],24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
4,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN


In [3]:
# Checking column headers
print(metadata_df.columns)

Index(['type', 'start_time', 'ambient_temperature', 'battery_id', 'test_id',
       'uid', 'filename', 'Capacity', 'Re', 'Rct'],
      dtype='object')


In [4]:
# Filtering out dataset with type impedance and also creating original dataframe of dataset
impedance_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'Re', 'Rct', 'battery_id']]
dataset_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'filename', 'battery_id']]

In [5]:
# Converting start time to appropriate format
from datetime import datetime

def parse_start_time(value):
    try:
        if isinstance(value, str):
            value = value.strip("[]").replace(",", "")
            components = [float(x) for x in value.split()]
            if len(components) == 6:
                year, month, day, hour, minute = map(int, components[:5])
                second = int(components[5])
                return datetime(year, month, day, hour, minute, second)
    except (ValueError, SyntaxError, TypeError):
        return pd.NaT
    return pd.NaT


In [6]:
dataset_df['start_time'] = dataset_df['start_time'].apply(parse_start_time)
impedance_df['start_time'] = impedance_df['start_time'].apply(parse_start_time)

# Dropping null values and sorting based on start time
impedance_df = impedance_df.dropna(subset=['start_time'])
impedance_df = impedance_df.sort_values(by='start_time')

dataset_df = dataset_df.dropna(subset=['start_time'])
dataset_df = dataset_df.sort_values(by='start_time')
impedance_df['battery_id'].unique()

array(['B0005', 'B0007', 'B0006', 'B0018', 'B0028', 'B0027', 'B0025',
       'B0026', 'B0029', 'B0031', 'B0030', 'B0032', 'B0036', 'B0034',
       'B0033', 'B0039', 'B0038', 'B0040', 'B0043', 'B0044', 'B0042',
       'B0041', 'B0047', 'B0045', 'B0046', 'B0048', 'B0052', 'B0049',
       'B0050', 'B0051', 'B0054', 'B0056', 'B0055', 'B0053'], dtype=object)

In [7]:
#Filtering out information for each battery_id and plotting curves
for battery_id in impedance_df['battery_id'].unique() :
    current_battery_info = impedance_df[impedance_df['battery_id'] == battery_id]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=current_battery_info['start_time'], 
        y=current_battery_info['Re'], 
        mode='lines', 
        name='Re',
        line=dict(color='green')
    ))
    fig.add_trace(go.Scatter(
        x=current_battery_info['start_time'], 
        y=current_battery_info['Rct'], 
        mode='lines', 
        name='Rct',
        line=dict(color='blue')
    ))
    fig.update_layout(
        title=f"Resistance Battery id {battery_id}",
        xaxis_title="Start Time",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(tickangle=0),
        legend_title="Legend",
        template="plotly"
    )
    import plotly.io as pi
    pi.renderers.default = 'iframe_connected'
    fig.show()

In [10]:
battery_info = {}
for idx, row in dataset_df.iterrows() :
    battery_id = row['battery_id']
    filename = row['filename']
    start_time = row['start_time']
    filePath = f"C:/Users/Neeraj kalantri/Downloads/cleaned_dataset/data/{filename}"
    try:
        data = pd.read_csv(filePath)
        
        if 'Battery_impedance' in data.columns:
            impedance_values = data['Battery_impedance'].apply(lambda x: complex(x).real)
            avg_impedance = impedance_values.mean()
            if battery_id not in battery_info:
                battery_info[battery_id] = {'start_time': [], 'battery_impedance': []}
            battery_info[battery_id]['start_time'].append(start_time)
            battery_info[battery_id]['battery_impedance'].append(avg_impedance)
    except Exception as e:
        print(f"Error occurred with {filename}: {e}")
for battery_id, data in battery_info.items():
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data['start_time'],
                                 y=data['battery_impedance'],
                                 mode='lines',
                                 name=f'Battery {battery_id}')
                     )
        fig.update_layout(
            title=f'Battery impedance battery id {battery_id}',
            xaxis_title='Start Time',
            yaxis_title='Battery Impedance (Ohms)',
            xaxis=dict(tickangle=0),
            template='plotly'
        )
        import plotly.io as pi
        pi.renderers.default = 'iframe_connected'
        fig.show()
